In [3]:
import os
import csv
import re
import pandas as pd
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokens import Doc, Span
from spacy.pipeline import Pipe
from collections import Counter

# Load the English language model
nlp = spacy.load("en_core_web_sm")

In [ ]:
# This code block gets all the comments in an unformatted way
# Useful for checking whats missing in the formatted way below
def get_comments(filepath):
  comments = []
  with open(filepath, 'r') as f:
    for line in f:
      line = line.strip()
      if line.startswith('#'):
        comments.append(line)
      else:
        break 
  return comments

def find_gpt_files(directory):
  gpt_files = []
  for root, _, files in os.walk(directory):
    for file in files:
      if file == 'gpt.py':
        gpt_files.append(os.path.join(root, file))
  return gpt_files

def save_comments_to_csv(comments, filename):
  with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for comment in comments:
      writer.writerow([comment])  

def main():
  kattis_dir = os.path.join(os.getcwd(), 'kattis')
  gpt_files = find_gpt_files(kattis_dir)
  all_comments = []

  for filepath in gpt_files:
    comments = get_comments(filepath)
    all_comments.extend(comments)

  if all_comments:
    csv_filename = 'comments.csv'
    save_comments_to_csv(all_comments, csv_filename)
    print(f"Comments saved to CSV file: {csv_filename}")
  else:
    print("No comments found in any gpt.py files.")

if __name__ == "__main__":
  main()


In [ ]:
# This code block gives the comments in a formatted way which lets us convert to a dataframe
def get_comments(filepath):
    comments = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('#'):
                comments.append(line)
            else:
                break 
    return comments

def extract_info(comment):
    # Regular expression to extract difficulty, result, and tests from comments
    pattern = r'# (\d+\.\d+) (\w+ [\w\s-]+) (\d+/\d+)'
    match = re.match(pattern, comment)
    if match:
        return match.groups()
    
    pattern_accepted = r'# (\d+\.\d+) (\w+) (\d+/\d+)' 
    match_accepted = re.match(pattern_accepted, comment)
    if match_accepted:
        return match_accepted.groups()
    return None

def find_gpt_files(directory):
    gpt_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file == 'gpt.py':
                gpt_files.append(os.path.join(root, file))
    return gpt_files

def save_comments_to_csv(comments, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Difficulty", "Result", "Tests"])
        for comment in comments:
            info = extract_info(comment)
            if info:
                writer.writerow(info)

def main():
    kattis_dir = os.path.join(os.getcwd(), 'kattis')
    gpt_files = find_gpt_files(kattis_dir)
    all_comments = []

    for filepath in gpt_files:
        comments = get_comments(filepath)
        all_comments.extend(comments)

    if all_comments:
        csv_filename = 'kattis_comments.csv'
        save_comments_to_csv(all_comments, csv_filename)
        print(f"Comments saved to CSV file: {csv_filename}")
    else:
        print("No comments found in any gpt.py files.")

if __name__ == "__main__":
    main()

In [ ]:
# Creating a dataframe from the comments csv
df_kattis = pd.read_csv("./kattis_comments.csv")
df_kattis['Result'] = df_kattis['Result'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df_kattis

In [ ]:
# Creating more columns for splitting the tests 
df_kattis[['Correct', 'Total']] = df_kattis['Tests'].str.split('/',expand=True).astype(int)
df_kattis

In [ ]:
# Getting the count of each result at the difficulties
grouped = df_kattis.groupby(['Difficulty','Result']).size().unstack(fill_value=0)
grouped

In [ ]:
# Creating a stacked bar chart for each result at each difficulty
ax = grouped.plot(kind='bar', stacked=True, figsize=(10, 6))
ax.set_xlabel('Difficulty')
ax.set_ylabel('Count')
ax.set_title('Outcome Count by Difficulty Level')
plt.xticks(rotation=45)
plt.legend(title='Outcome')
plt.show()

# Figure out which feutures to have as inputs in a prediction model, Figure engineering. With what to get from the problem statements. Readability, frequency of words, etc. think of meaningful features. perform unsupervised learning, understanding the features.

In [4]:
# Functions to get all problems
def find_prompt_files(directory):
    prompt_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if 'gpt_prompt' in file:  # Check if 'gpt_prompt' is in the file name
                prompt_files.append(os.path.join(root, file))
    return prompt_files

def get_problem(filepath):
    with open(filepath, 'r') as f:
        problem = f.read().strip()
    return [problem]

kattis_dir = os.path.join(os.getcwd(), 'kattis')
prompt_files = find_prompt_files(kattis_dir)
all_problems = []

for filepath in prompt_files:
    comments = get_problem(filepath)
    all_problems.extend(comments)


In [6]:
all_problems = all_problems[:20]
for problem in all_problems:
        doc = nlp(problem)

        # Extract keywords (nouns and verbs) excluding stopwords
        keywords = [token.text.lower() for token in doc if token.text.lower() not in STOP_WORDS]

        # Count the frequency of each keyword
        keyword_freq = Counter(keywords)

        # Print the top keywords and their frequencies
        print(keyword_freq)

['2048', 'single', '-', 'player', 'puzzle', 'game', 'created', 'gabriele', 'cirulli1', '.', 'played', '4x4', ' ', 'grid', 'contains', 'integers', '>', '=', '2', 'powers', '2', '.', 'player', 'use', 'keyboard', 'arrow', 'key', '(', 'left', '/', '/', 'right', '/', ')', 'tiles', 'simultaneously', '.', 'tiles', 'slide', 'far', 'possible', 'chosen', 'direction', 'stopped', 'tile', 'edge', 'grid', '.', 'tiles', 'number', 'collide', 'moving', ',', 'merge', 'tile', 'total', 'value', 'tiles', 'collided', '.', 'resulting', 'tile', 'merge', 'tile', '.', 'observe', 'merging', 'behavior', 'carefully', 'sample', 'inputs', 'outputs', '.', '\n\n', 'input', '\n', 'input', 'valid', 'game', 'state', '2048', 'puzzle', '.', 'lines', 'input', ',', 'contains', 'integers', ',', 'describe', '16', 'integers', '4x4', 'grid', '2048', 'puzzle', '.', 'j', '-', 'th', 'integer', '-', 'th', 'line', 'denotes', 'content', 'cell', 'located', '-', 'th', 'row', 'j', '-', 'th', 'cell', '.', 'problem', ',', 'integers', 'inpu